# World Coordinate Systems in Astropy

__Author: Kaustubh Vaghmare__

If you open an image is ds9 or any other image viewer, you will notice that as the mouse pointer moves across various regions in the image, a part of the window will reflect the current coordinates in some coordinate system. This is possible because the image header contains a whole bunch of keywords which can be used to compute the coordinates as a function of the pixel coordinates.

What one needs is the coordinate for one pixel and then variation in coordinates for every pixel of movement. But if the image is very large, the curvature of the sky can become important. Then additional information to account for second order effects will need to be recorded

Let's see how a good fraction of all this is handled using Astropy.

Simple steps:

    Get hold of the header.
    Pass it to a WCS object constructor.
    And use this object for coordinate conversions.


In [4]:
from astropy.io import fits
import astropy.wcs as wcs

hdulist = fits.open('data/example2.fits')
w = wcs.WCS(hdulist[0].header)
print(w)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 205.4864335  47.26686298333333  
CRPIX : -3929.556588388735  377.0604456526553  
PC1_1 PC1_2  : 0.02528041441746252  2.266833305288208e-05  
PC2_1 PC2_2  : -1.983416356621748e-05  0.02528846844193638  
CDELT : -0.01868178756738699  0.01868178756738699  
NAXIS : 777  777


In [5]:
# Convert pixels to coordinates.
w.wcs_pix2world(100, 100, 1)

[array(202.69037279), array(47.10039636)]

In [6]:
w.wcs_pix2world([100, 105], [100, 105], 1)

[array([202.69037279, 202.68677899]), array([47.10039636, 47.10266995])]

In [7]:
w.wcs_world2pix(202.69, 47.1, 1)

[array(100.56778084), array(99.1810579)]

In [8]:
w.calc_footprint()

array([[202.76146626,  47.05535561],
       [202.74206475,  47.42164629],
       [202.2009436 ,  47.40718098],
       [202.22406523,  47.04102483]])

## Some convenience functions

There are some utilities provided for your convenience which make coordinate to pixel and vice versa calculations much simple.

In [9]:
output = wcs.utils.pixel_to_skycoord(100, 100, w)
output

<SkyCoord (FK5: equinox=2000.0): (ra, dec) in deg
    (202.68965405, 47.10085109)>

Remember: This is capable of ignoring higher dimensions in WCS in radio data, for example.

The output here is a special object known as a sky coordinate object. This can be created by saying,

    from astropy.coordinates import SkyCoord
    c = SkyCoord(10, 20, unit='deg')



In [11]:
import astropy

new = output.transform_to(astropy.coordinates.Galactic)
print(new)

<SkyCoord (Galactic): (l, b) in deg
    (104.37183205, 68.58518637)>


In [12]:
from astropy.coordinates import SkyCoord

distance = output.separation(SkyCoord(202, 47, unit='deg'))
print(distance)

0d28m50.1524s


In [13]:
print(output.ra, output.dec)

202d41m22.7546s 47d06m03.0639s
